In [1]:
#in: build X with utils/function build_X('test|train')
#out: metrics(acc, recall, prec), roc_curve, log_loss
#Google Colab needs
import os
import sys
def isCollab():
    return os.environ.get('COLAB_GPU', None) != None

if isCollab():
    #Mounting GDrive disc
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_storage = '/content/gdrive/My Drive/UCU-2019-final-project-storage'

    #Append path where custom modules stored. I put custom modules to GDrive disc
    path_to_modules = '/content/gdrive/My Drive/UCU-2019-final-project-storage'
    sys.path.append(path_to_modules)
else:
    sys.path.append('..')
    path_to_storage = None

In [2]:
import keras
import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
import re
from keras import optimizers
from keras import activations
from keras import losses
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras import layers
from keras import activations
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Input
import gensim
from keras.models import Model
import keras.backend as K
import nltk

Using TensorFlow backend.
/Users/denisporplenko/anaconda3/envs/ucu-2019-ml-final-project/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/denisporplenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
if not path_to_storage:
    path_to_storage = os.path.abspath(os.path.join(os.getcwd(), '../storage')) 

path_to_objects = path_to_storage + "/serialization_objects/"
data_folder = path_to_storage+'/data/'
X_train = pickle.load(open(path_to_objects + 'X_train.p', 'rb'))
y_train = pickle.load(open(path_to_objects + 'y_train.p', 'rb'))
X_test = pickle.load(open(path_to_objects + 'X_test.p', 'rb'))
y_test = pickle.load(open(path_to_objects + 'y_test.p', 'rb'))

In [0]:
max_seq_length = max(X_train.question1.map(lambda x:len(x)).max(),
                    X_train.question2.map(lambda x:len(x)).max(),
                    X_test.question1.map(lambda x:len(x)).max(),
                    X_test.question2.map(lambda x:len(x)).max())

In [7]:
#Download GoogleNews-vectors-negative300.bin
path_to_google_news_model = data_folder+'GoogleNews-vectors-negative300.bin'
if not os.path.isfile(path_to_google_news_model):
    !wget -P "$data_folder" -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz" 
    

In [9]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  

word2vec = gensim.models.KeyedVectors.load_word2vec_format(path_to_google_news_model, binary=True)

In [0]:
def text_to_word_list(text):
    ''' 
    Pre process and convert texts to a list of words 
    input: str
    output: list of cleaned word
    '''
    
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub("quikly","quickly", text)

    text = text.split()

    return text

In [31]:
questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [X_train, X_test]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions with lists of word indices
            dataset.set_value(index, question, q2n)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [0]:
embedding_dim = 300

embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  
embeddings[0] = 0 


# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

In [0]:
del word2vec

In [0]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

  
n_hidden = 50
gradient_clipping_norm  = 1.25

batch_size = 64
n_epoch = 3
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = layers.Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

shared_lstm = keras.layers.LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

distance = keras.layers.Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])


model = Model([left_input, right_input], [distance])

In [0]:
from keras import optimizers
optimizer = optimizers.Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
import itertools
# Split to train validation
validation_size = 40000
training_size = len(X_train) - validation_size

X = X_train[questions_cols]
Y = y_train

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}

Y_train = Y_train.values
Y_validation = Y_validation.values

for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side],  maxlen=max_seq_length)

In [37]:
trained = model.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 230872 samples, validate on 40000 samples
Epoch 1/3
230872/230872 [==============================] - 11446s 50ms/step - loss: 0.1806 - acc: 0.7353 - val_loss: 0.1643 - val_acc: 0.7633
Epoch 2/3
230872/230872 [==============================] - 11341s 49ms/step - loss: 0.1588 - acc: 0.7743 - val_loss: 0.1545 - val_acc: 0.7830
Epoch 3/3
230872/230872 [==============================] - 11317s 49ms/step - loss: 0.1506 - acc: 0.7893 - val_loss: 0.1498 - val_acc: 0.7890


In [0]:
pickle.dump(trained, open(path_to_objects+"lstm_trained.p", 'wb'))